## AUC Calculation per Series and Volume from CSV Files

This script automates the computation of **AUC (Area Under the Curve)** values for sensitivity vs. precision data across different MRI series and volumes. It is designed for analyzing segmentation performance, where each CSV file contains results for a specific method and brain region.

### What the script does:

1. **Input**:  
   - It processes a list of CSV files (e.g., `Sato_prueba_BG.csv`, `Sato_prueba_CSO.csv`, etc.) which contain columns like `SerieName`, `VolumeName`, `sensitivity`, and `precision`.

2. **AUC Calculation**:
   - For each unique combination of `SerieName` and `VolumeName`, the script:
     - Sorts the data by sensitivity.
     - Computes the AUC between sensitivity and precision using the trapezoidal rule.
   - All AUC values are stored and exported.

3. **Best AUC per Series**:
   - For each `SerieName`, the volume with the **maximum AUC** is selected and saved separately.

4. **Output**:
   - Two CSV files are created per input file:
     - One with **all AUC values** per volume.
     - One with the **maximum AUC value per series**.

This is useful for comparing algorithm performance across different anatomical regions and identifying the most accurate segmentation results.

In [ ]:
import pandas as pd
from sklearn.metrics import auc
import os

def calculate_max_auc_and_save(df, max_output_csv, all_output_csv):
    # Ensure 'sensitivity' and 'precision' columns are numeric
    df['sensitivity'] = pd.to_numeric(df['sensitivity'], errors='coerce')
    df['precision'] = pd.to_numeric(df['precision'], errors='coerce')

    # Get unique SerieNames
    serie_names = df['SerieName'].unique()

    # List to store AUC results
    auc_results = []

    # Calculate AUC for each VolumeName within each SerieName
    for serie_name in serie_names:
        # Filter by SerieName
        filtered_df = df[df['SerieName'] == serie_name]

        # Get unique VolumeNames
        volume_names = filtered_df['VolumeName'].unique()
        for volume_name in volume_names:
            volume_df = filtered_df[filtered_df['VolumeName'] == volume_name]

            # Sort by sensitivity before computing AUC
            sorted_df = volume_df.sort_values(by='sensitivity')
            auc_value = auc(sorted_df['sensitivity'], sorted_df['precision'])

            # Store result
            auc_results.append({
                'SerieName': serie_name,
                'VolumeName': volume_name,
                'AUC': auc_value
            })

    # Create DataFrame with all AUC results
    auc_df = pd.DataFrame(auc_results)

    # Save all AUC results
    auc_df.to_csv(all_output_csv, index=False)

    # Get the row with the max AUC per SerieName
    max_auc_per_serie = auc_df.loc[auc_df.groupby('SerieName')['AUC'].idxmax()]

    # Save max AUC results
    max_auc_per_serie.to_csv(max_output_csv, index=False)

    print(f"Max AUC results saved to {max_output_csv}")
    print(f"All AUC results saved to {all_output_csv}")

# List of CSV files
csv_files = ['Sato_prueba_BG.csv', 'Sato_prueba_CSO.csv', 'Sato_prueba_Organ.csv']

# Process each CSV file
for csv_file in csv_files:
    # Load CSV
    df = pd.read_csv(csv_file)

    # Extract suffix from filename
    base_name = os.path.basename(csv_file)
    name_suffix = base_name.split('_')[2].split('.')[0]

    # Define output filenames
    max_output_csv = f'{name_suffix}_Max_AUC_Results.csv'
    all_output_csv = f'{name_suffix}_All_AUC_Results.csv'

    # Calculate and save AUC results
    calculate_max_auc_and_save(df, max_output_csv, all_output_csv)

Max AUC results saved to BG_Max_AUC_Results.csv
All AUC results saved to BG_All_AUC_Results.csv
Max AUC results saved to CSO_Max_AUC_Results.csv
All AUC results saved to CSO_All_AUC_Results.csv
Max AUC results saved to Organ_Max_AUC_Results.csv
All AUC results saved to Organ_All_AUC_Results.csv
